# 1. Quantum computer

In all this worksheet, we will use an emulator in Sage of a quantum computer.

For this, we stary by importing the relevant package

In [1]:
from quantum.computer import QuantumComputer


## Instantiating a quantum computer

To use this package, we first need to instantiate a quantum computer; this is done as follows

In [2]:
QC = QuantumComputer()
QC

Quantum computer

## Managing memory

The method `malloc(n)` allocates a range of memory of `n` qubits and returns a pointer to it.

In [3]:
x = QC.malloc(2)
x

Quantum register of 2 qubits

In our implementation, the type of a register can only be a pointer to an `unsigned int`, *i.e.* a register of `n` qubits stores an integer between $0$ and $2^n-1$. At the creation, all variables are initialized with the value `0`.

The method `used_memory` returns the number of allocated qubits.

In [4]:
QC.used_memory()

2

In [5]:
a = QC.malloc(1)
b = QC.malloc(1)
c = QC.malloc(1)

In [6]:
QC.used_memory()

5

It is also possible to desallocate memory using the method `free`.

*However, you should be very careful with this because freeing memory first performs a measure and so may alter other qubits.*

In [7]:
QC.free(c)

0

In [8]:
QC.used_memory()

4

## Builtin Gates

Our implementation includes a few builtin gates.

**The Hadamard gate**

The following syntax apply to Hadamard gate to each bit of `x`

In [9]:
QC.hadamard(x)

This function does not return anything but modify the internal state of the quantum computer (creating here a uniform superposition of all possible value in the register `x` and letting unchanged all other registers). We can have access to it using the method `internal_state()`.

**Caution**: the values returned by this method are not accessible in the real world; as a consequence, this function is only meant for pedogogical purpose and debugging. Don't use it in your programs!

In [10]:
QC.internal_state()

0.500000000000000*|0000> + 0.500000000000000*|0100> + 0.500000000000000*|1000> + 0.500000000000000*|1100>

**The NOT Gate**

Similarly to the Hadamard case, the following syntax applies the NOT gate to each bit of `x`

In [11]:
QC.X(x)

**The controlled NOT gate**

In the following syntax, the controlling bit is the first register.

In [12]:
QC.CX(a, x)

**The double controlled NOT gate (Toffoli gate)**

In the following syntax, the controlling bits are the two first registers.

In [13]:
QC.CCX(a, b, x)

After applying all the previous gates, the internal state is now:

In [14]:
QC.phase_shift(x, pi/8)

In [15]:
QC.internal_state()

0.500000000000000*|0000> + (0.461939766255643 + 0.191341716182545*I)*|0100> + (0.461939766255643 + 0.191341716182545*I)*|1000> + (0.353553390593274 + 0.353553390593274*I)*|1100>

## Oracles: gates associated to classical functions

The method `apply(f, x, y)` applies the gate of the form $U_f$ where $f$ is a usual function acting on integers.

In [16]:
def mod2(v):
    return v % 2

In [17]:
QC.apply(mod2, x, a)

In [18]:
QC.internal_state()

0.500000000000000*|0000> + (0.461939766255643 + 0.191341716182545*I)*|0110> + (0.461939766255643 + 0.191341716182545*I)*|1000> + (0.353553390593274 + 0.353553390593274*I)*|1110>

This method also works for multivariate functions.

The two syntaxes `QC.apply(function, x1, x2, ..., xn, y)` or `QC.apply(function, [x1, x2, ..., xn], y)` are allowed.

**Exercise 1.1**: initiate a new quantum computer and create the superposition:
$$\frac 1 4 \cdot \sum_{i=0}^3 \sum_{j=0}^3 \left|i\right> \otimes \left|j\right> \otimes \left|i+j\right>$$
where $i$ and $j$ are encoded on the register of 2 qubits and the result $i+j$ is encoded on the register of 3 qubits.

In [25]:
QC = QuantumComputer()
x = QC.malloc(2)  # register for i's
y = QC.malloc(2)  # register for j's
z = QC.malloc(3)  # register for the sum
QC.hadamard(x)
QC.hadamard(y)
def add(x,y):
    return x+y
QC.apply(add, x,y,z)
QC.internal_state()

0.250000000000000*|0000000> + 0.250000000000000*|0001001> + 0.250000000000000*|0010010> + 0.250000000000000*|0011011> + 0.250000000000000*|0100001> + 0.250000000000000*|0101010> + 0.250000000000000*|0110011> + 0.250000000000000*|0111100> + 0.250000000000000*|1000010> + 0.250000000000000*|1001011> + 0.250000000000000*|1010100> + 0.250000000000000*|1011101> + 0.250000000000000*|1100011> + 0.250000000000000*|1101100> + 0.250000000000000*|1110101> + 0.250000000000000*|1111110>

## Measures

The method `measure(x)` measures all the bits of the register `x` and returns the measured value (as an integer).

In [26]:
QC.measure(z)

3

**Exercise 1.2**: what are the possible outputs of the command above? and what are the probabilites to get them?

As required by the theory, the measure alters the internal state of the computer.

In [27]:
QC.internal_state()

0.500000000000000*|0011011> + 0.500000000000000*|0110011> + 0.500000000000000*|1001011> + 0.500000000000000*|1100011>

## Complexity

The method `cputime` counts the number of elementary gates and measures which were applied since the instantation of this quantum computer. Oracles (gates associated to classical functions) are not counted and Hadamard gates, NOT gates and measures acting on `n` qubits count for `n`.

In [28]:
QC.cputime()

7

The method `walltime` does the same counting but includes oracles.

In [29]:
QC.walltime()

8

# 2. Quantum adder

In [32]:
def QuantumSemiadder(QC, a, b, cin, cout):
    QC.CCX(a, b, cout)
    QC.CX(a, b)
    QC.CCX(b, cin, cout)
    QC.CX(b, cin)
    QC.CX(a, b)

**Exercise 3.1**: write the circuit correspoing to the previous code and observe that it implements a semi-adder: after it is executed the number `cout cin` (in base 2) is the sum `a + b + cin`

**Exercice 3.2**: design a circuit acting on $(3n+1)$-qubits with the property that:

$$\left|x\right> \otimes \left|y\right> \otimes \left|0\right> \mapsto \left|x\right> \otimes \left|y\right> 
\otimes \left|x+y\right>$$

where $x$ and $y$ are two integers with $n$ digits and the plus sign is the classical addition.

**Exercice 3.3**: write a function `QuantumAdder(q)` implementing this circuit

In [45]:
def QuantumAdder(QC,q):
    #supposes q is |x>|y>|0> of size 3n+1
    l = (q.size()-1)//3
    for i in range(n):
        QuantumSemiadder(QC,q[n-1-i],q[2*n-1-i],q[3*n-1-i], q[3*n-i])

**Exercice 3.4**: write tests demonstrating that your function `QuantumAdder` works correctly

In [54]:
QC_=QuantumComputer()
n=3

q=QC_.malloc(3*n+1)

QC_.hadamard(q[0])

QC_.internal_state()
QuantumAdder(QC_,q)
QC_.internal_state()

0.707106781186548*|0000000000> + 0.707106781186548*|0001000001>